## Turborreactor en vuelo FL100/250KTS

Se dispone de un turborreactor a $FL100$ (nivel de vuelo $10000ft$) volando en un vehículo a $v_0=250KTS$. El turborreactor toma $60\frac{kg}{s}$ por su difusor, en el que se conoce que la presión y temperatura estáticas ($p_1, T_1$) son como las de la atmósfera estándar internacional.

El turborreactor tiene una arquitectura conocida compuesta por un difusor isoentrópico, dos compresores de rendimientos conocidos ($\eta_{c1}=0.8$,  $\eta_{c2}=0.75$) y de relación de compresión 3:1 y 5:1 respectivamente, una cámara de combustión sin pérdida de carga y rendimiento de combustión del 90\% y una única turbina que acopla a ambos compresores de rendimiento 90\%. El turborreactor descarga por una tobera de la que se sabe que que es isoentrópica y tiene un área de $A_9=0.2063m^2$.

El combustible empleado es un hidrocarburo de dosado $f=0.035$ y poder calorífico $L=42\frac{MJ}{kg}$.

Se pide:
1. Calcular Mach de la corriente libre y área de entrada al turborreactor
2. Obtener la presión y temperatura de remanso en todas las estaciones
3. Resolver la tobera y calcular la velocidad de eyección de la corriente
4. Empuje, impulso, consumo específico y rendimiento

Nota: La resolución numérica de este notebook requiere de `pyTurb` (https://github.com/MRod5/pyturb) instalado para su ejecución.

In [2]:
from pyturb.gas_models import PerfectIdealGas
from pyturb.gas_models import IsentropicFlow
from pyturb.gas_models import isa
from pyturb.utils import units
import numpy as np

In [3]:
aire = PerfectIdealGas('Air')
aire_isentropico = IsentropicFlow(aire)

In [4]:
G1 = 60 #kg/s
h1 = units.ft_to_m*10000
v1 = units.kts_to_ms*250

pic1 = 3
pic2 = 5
eta_c1 = 0.8
eta_c2 = 0.75
eta_q = 0.9
eta_t = 0.9

f = 0.035
L = 42e6 #J/kg
A9 = 0.2063 #m^2

In [5]:
T1 = isa.temperature_isa(h1)
p1 = isa.pressure_isa(h1)
M1 = aire_isentropico.mach_number(v1,T1)
T1t = aire_isentropico.stag_temp_from_mach(M1, T1)
p1t = aire_isentropico.stag_pressure_from_mach(M1, p1)
rho1 = p1/aire.Rg/T1
A1 = G1/rho1/v1
H1t = G1*aire.cp()*T1t

print(' h1={:6.0f}m;     v1={:6.1f}m/s;  M1={:6.3f}[-]'.format(h1, v1, M1))
print(' A1={:6.4f}m^2;   p1={:6.0f}m/s;  T1={:6.1f}K;  rho1={:5.3f}M^2'.format(A1, p1, T1, rho1))
print('p1t={:6.0f}Pa;   T1t={:6.1f}K;   H1t={:6.2f}MW'.format(p1t, T1t, H1t*1e-6))

 h1=  3048m;     v1= 128.6m/s;  M1= 0.392[-]
 A1=0.5157m^2;   p1= 69682m/s;  T1= 268.3K;  rho1=0.905M^2
p1t= 77455Pa;   T1t= 276.6K;   H1t= 16.67MW


In [6]:
G3 = G25 = G2 = G1
T2t = T1t # Difusor
p2t = p1t # Isoentrópico
H2t = H1t # Difusor

p25t = p2t * pic1
p3t = p25t * pic2
T25t = ((pic1**((aire.gamma()-1)/aire.gamma()) - 1) / (eta_c1) + 1) * T2t
T3t = ((pic2**((aire.gamma()-1)/aire.gamma()) - 1) / (eta_c2) + 1) * T25t
H25t = G25*aire.cp()*T25t
H3t = G3*aire.cp()*T3t

Wc1 = H25t - H2t
Wc2 = H3t - H25t
print(' p2t={:8.0f}Pa;  T2t={:6.1f}K;  H2t={:6.2f}MW'.format(p2t, T2t, H2t*1e-6))
print('p25t={:8.0f}Pa; T25t={:6.1f}K; H23t={:6.2f}MW'.format(p25t, T25t, H25t*1e-6))
print(' p3t={:8.0f}Pa;  T3t={:6.1f}K;  H3t={:6.2f}MW'.format(p3t, T3t, H3t*1e-6))

 p2t=   77455Pa;  T2t= 276.6K;  H2t= 16.67MW
p25t=  232364Pa; T25t= 404.0K; H23t= 24.36MW
 p3t= 1161822Pa;  T3t= 718.5K;  H3t= 43.32MW


In [7]:
p4t = p3t
c = f*G3
G4 = G3 + c
Q43 = eta_q*c*L
H4t = H3t + Q43
T4t = H4t/G4/aire.cp()
print(' p4t={:8.0f}Pa;  T4t={:6.1f}K;  H4t={:6.2f}MW'.format(p4t, T4t, H4t*1e-6))

 p4t= 1161822Pa;  T4t=1966.5K;  H4t=122.70MW


In [8]:
G5 = G4
Wt = -(Wc1+Wc2)
H5t = Wt + H4t
T5t = H5t/G5/aire.cp()
p5t = ((T5t/T4t - 1)/eta_t + 1)**(aire.gamma()/(aire.gamma()-1)) * p4t
print(' p5t={:8.0f}Pa;  T5t={:6.1f}K;  H5t={:6.2f}MW'.format(p5t, T5t, H5t*1e-6))

 p5t=  441990Pa;  T5t=1539.5K;  H5t= 96.05MW


In [9]:
G9 = G8 = G7 = G5
p9t = p8t = p7t = p5t
T9t = T8t = T7t = T5t
H9t = H8t = H7t = H5t

In [10]:
itera = True
n_iter = 0
iter_max = 100
tolerancia = 1e-4
T9_ = T1
while itera:
    p9_ = G9*aire.Rg*T9_ / (A9 * np.sqrt(2*aire.cp()*(T9t-T9_)))
    T9 = T9t/((p9t/p9_)**((aire.gamma()-1)/aire.gamma()))
    residuo = np.abs(T9-T9_)
    print(n_iter, p9_, T9, residuo)
    if residuo <= tolerancia:
        print()
        itera = False
    elif n_iter>= iter_max:
        print()
        itera = False
    else:
        n_iter = n_iter + 1
        T9_ = T9
        itera = True
p9 = p9_
v9 = np.sqrt(2*aire.cp()*(T9t-T9))
print('v9={:6.1f}m/s'.format(v9))

0 14507.72127206873 580.0211177122299 311.6831177122299
1 36094.83008740942 752.5541890640704 172.5330713518406
2 51711.311097508165 833.9642966092018 81.41010754513138
3 60521.40011860219 872.3034724056416 38.339175796439804
4 65097.179933921594 890.6580326499857 18.354560244344043
5 67400.5161836193 899.5501698938874 8.892137243901743
6 68544.75768699123 903.8870502996247 4.336880405737247
7 69109.80624389459 906.0096328800346 2.122582580409926
8 69388.05536763836 907.0503120301902 1.0406791501555972
9 69524.8897925367 907.5609931590415 0.5106811288512745
10 69592.13664520865 907.8117027355779 0.2507095765364511
11 69625.17430913608 907.9348102981619 0.12310756258398214
12 69641.40281358092 907.9952669563204 0.06045665815850043
13 69649.37383936043 908.0249580333359 0.029691077015513656
14 69653.28885506933 908.0395400898213 0.014582056485437533
15 69655.21170212895 908.0467018044116 0.007161714590210977
16 69656.15609351489 908.0502191731495 0.0035173687379028706
17 69656.6199219263

In [11]:
E = G9*v9 - G1*v1 + A9*(p9-p1)
Isp = E/G9
ce = c/E
eta_m = (G9*v9**2 - G1*v1**2)/(2*c*L)
eta_p = 2*v1*(G9*v9-G1*v1)/(G9*v9**2 - G1*v1**2)
eta_mp = eta_m*eta_p

print('E={:6.2f}kN;  Isp={:6.1f}m/s;  ce={:6.4f}g/N/s'.format(E*1e-3, Isp, ce*1e3))
print('eta_m={:5.3f}[-]; eta_p={:5.3f}[-];  eta_mp={:5.3f}[-]'.format(eta_m, eta_p, eta_mp))

E= 62.23kN;  Isp=1002.1m/s;  ce=0.0337g/N/s
eta_m=0.441[-]; eta_p=0.206[-];  eta_mp=0.091[-]
